# EDA - HDB Resale

## Model Development - Tree-based Modeling

### Import Environment Modules and Data

In [102]:
### Import Python Modules

## System Modules
import sys
import os
import warnings
import logging
import time
import random
from typing import Union, Optional


## Essential Modules
import numpy as np
import pandas as pd


## Pipeline and Train Test Split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## SciKit Learning Preprocessing  
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, PowerTransformer
from sklearn.preprocessing import OneHotEncoder

### SciKit Learn ML Models

## Linear Models
from sklearn.linear_model import LinearRegression, HuberRegressor, QuantileRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.compose import TransformedTargetRegressor

## Tree Based Linear Models
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

## Performance Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score


### Hyper-parameter Tuning

## GridSearch CV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

## explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingRandomSearchCV # type: ignore

## Optuna for Hyperparameter Tuning
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntDistribution, FloatDistribution


## Problem Statement

**We work for a property consultancy company. This company want to develop an end to end machine learning pipeline that could deliver housing price prediction to customer.**

**Our task is to develop a machine learning model that could accurately predict the resale prices of HDB resale flats. This model will assist buyers or sellers in planning their budgets more effectively and set realistic expectations. This model also need to help buyers determine the type of flat they can afford and in which location. This model also should provide sellers with valuable information regarding the potential market value of their property.**

## Data Preprocessing For Model Building

In [103]:
# Load the dataset
df = pd.read_csv('./data/hdb_resale_price.csv')

In [104]:
len(df)

211086

### Removing Duplicates

In [105]:
# Remove duplicated items
df.drop_duplicates(inplace=True)

In [106]:
df.duplicated().sum()

np.int64(0)

In [107]:
len(df)

210782

In [108]:
df.describe()

,floor_area_sqm,lease_commence_date,resale_price
count,210782.000000,210782.000000,2.107820e+05
mean,96.860812,1996.336770,5.176226e+05
std,24.038202,14.241823,1.828416e+05
min,31.000000,1966.000000,1.400000e+05
25%,82.000000,1985.000000,3.800000e+05
50%,93.000000,1996.000000,4.850000e+05
75%,112.000000,2011.000000,6.200000e+05
max,366.700000,2022.000000,1.658888e+06


## Feature Engineering

### Feature Engineering - Splitting Transaction Column 'month' to Year and Month

In [109]:
def convert_month_to_year_month(df: pd.DataFrame, 
                              month: str = 'month',
                              transaction_year: str = 'transaction_year',
                              transaction_month: str = 'transaction_month') -> pd.DataFrame:
    """
    Convert month column to separate year and month columns.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe containing the month column
    month : str, default 'month'
        Name of the column containing month data
    transaction_year : str, default 'transaction_year'
        Name of the output year column
    month_out_col : str, default 'transaction_month'
        Name of the output month column
    
    Returns:
    --------
    pd.DataFrame
        Modified dataframe
    
    Raises:
    -------
    ValueError
        If month_col doesn't exist in the dataframe
    TypeError
        If df is not a pandas DataFrame
    """
    
    # Input validation
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame")
    
    # Check if dataframe is empty
    if df.empty:
        warnings.warn("Input dataframe is empty", UserWarning)
        return df.copy()
    
    # Check if month column exists
    if month not in df.columns:
        raise ValueError(f"Column '{month}' not found in dataframe")
    
    # Work on copy
    df = df.copy()
    
    # Convert to datetime with error handling
    try:
        df[month] = pd.to_datetime(df[month], format='%Y-%m', errors='coerce')
    except (ValueError, TypeError) as e:
        raise ValueError(f"Cannot convert column '{month}' to datetime: {str(e)}")
    
    # Extract year and month
    df[transaction_year] = df[month].dt.year
    df[transaction_month] = df[month].dt.month
    
    # Return dataframe
    return df

In [110]:
# =============================================================================
# SIMPLE TESTS - Run these in your notebook
# =============================================================================

def test_basic_functionality():
    """Test that the function works with normal data"""
    # Create test data
    test_df = pd.DataFrame({
        'month': ['2023-01', '2023-02', '2023-03'],
        'value': [100, 200, 300]
    })
    
    # Run function
    result = convert_month_to_year_month(test_df)
    
    # Check results
    assert 'transaction_year' in result.columns
    assert 'transaction_month' in result.columns
    assert result['transaction_year'].tolist() == [2023, 2023, 2023]
    assert result['transaction_month'].tolist() == [1, 2, 3]
    
    print("✓ Basic functionality test passed!")


test_basic_functionality()

✓ Basic functionality test passed!


In [111]:
def test_different_date_formats():
    """Test with different date formats"""
    test_df = pd.DataFrame({
        'month': ['2023-01', '2023-02', '2023-03'],
        'value': [100, 200, 300]
    })
    
    result = convert_month_to_year_month(test_df)
    
    assert result['transaction_year'].iloc[0] == 2023
    assert result['transaction_month'].iloc[0] == 1
    
    print("✓ Different date formats test passed!")

test_different_date_formats()

✓ Different date formats test passed!


In [112]:
def test_with_nulls():
    """Test with null values"""
    test_df = pd.DataFrame({
        'month': ['2023-01', None, '2023-03'],
        'value': [100, 200, 300]
    })
    
    result = convert_month_to_year_month(test_df)
    
    # First and third rows should work
    assert result['transaction_year'].iloc[0] == 2023
    assert result['transaction_month'].iloc[0] == 1
    
    print("✓ Null values test passed!")

test_with_nulls()

✓ Null values test passed!


In [113]:
def test_original_unchanged():
    """Test that original dataframe is not modified"""
    original_df = pd.DataFrame({
        'month': ['2023-01', '2023-02'],
        'value': [100, 200]
    })
    
    # Store original state
    original_columns = original_df.columns.tolist()
    
    # Run function
    result = convert_month_to_year_month(original_df)
    
    # Check original is unchanged
    assert original_df.columns.tolist() == original_columns
    assert 'transaction_year' not in original_df.columns
    
    # Check result has new columns
    assert 'transaction_year' in result.columns
    
    print("✓ Original unchanged test passed!")

test_original_unchanged()

✓ Original unchanged test passed!


In [114]:
# Splitting column 'month' into year and month
df = convert_month_to_year_month(df)

In [115]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,transaction_year,transaction_month
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017,1
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017,1
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017,1
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,2017,1
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,2017,1


### Feature Engineering - Convert 'remaining_lease' to remaining_lease_months'

In [116]:
def convert_lease_to_month(lease):
    """
    Convert remaining lease period from string to total number of months.
    Args:
        remaining_lease in (str)

    Returns: 
        integer

    Example:
        convert_lease_to_month('07 TO 09') -> 8.0  
    """
    str_list = lease.split(' ')
    if ('months' in str_list) | ('month' in str_list):
        year = int(str_list[0])
        month = int(str_list[2])
        t_month = (year * 12) + month 
    elif ('years' in str_list) & (('months' not in str_list) | ('month' not in str_list)):
        year = int(str_list[0])
        t_month = (year * 12)
    else:
        year = int(str_list[0])
        t_month = (year * 12)        
    return t_month

In [117]:
def test_convert_lease_to_month_simple():
    """Simple test cases for convert_lease_to_month function"""
    
    # Test years and months format
    assert convert_lease_to_month('5 years 6 months') == 66  # 5*12 + 6 = 66
    assert convert_lease_to_month('2 years 3 months') == 27  # 2*12 + 3 = 27
    assert convert_lease_to_month('1 years 0 months') == 12  # 1*12 + 0 = 12
    
    # Test years only format
    assert convert_lease_to_month('10 years') == 120  # 10*12 = 120
    assert convert_lease_to_month('5 years') == 60    # 5*12 = 60
    assert convert_lease_to_month('1 years') == 12    # 1*12 = 12
    
    # Test with singular 'month'
    assert convert_lease_to_month('3 years 1 month') == 37   # 3*12 + 1 = 37
    assert convert_lease_to_month('0 years 1 month') == 1    # 0*12 + 1 = 1
    
    # Test just numbers (should default to years)
    assert convert_lease_to_month('5') == 60    # 5*12 = 60
    assert convert_lease_to_month('2') == 24    # 2*12 = 24
    print("All simple test cases passed!")

test_convert_lease_to_month_simple()

All simple test cases passed!


In [118]:
def test_convert_lease_to_month_variations():
    """Test various input formats"""
    
    # Test different plural/singular combinations
    assert convert_lease_to_month('1 year 1 month') == 13    # Should work if 'year' -> 'years'
    assert convert_lease_to_month('10 years 11 months') == 131  # 10*12 + 11 = 131
    
    # Test zero cases
    assert convert_lease_to_month('0 years 6 months') == 6   # 0*12 + 6 = 6
    assert convert_lease_to_month('0 years') == 0      
    print("All variations test cases passed!")

test_convert_lease_to_month_variations()

All variations test cases passed!


In [119]:
# Convert column remaining lease to remaining lease by  months
df['remaining_lease_by_months'] = df.remaining_lease.apply(convert_lease_to_month)


In [120]:

df[['remaining_lease', 'remaining_lease_by_months']].sample(10)

,remaining_lease,remaining_lease_by_months
17788,72 years 03 months,867
50704,59 years 11 months,719
69147,82 years 01 month,985
77778,94 years 03 months,1131
84515,81 years 01 month,973
39163,93 years,1116
203251,82 years 07 months,991
53901,92 years 05 months,1109
105184,95 years 03 months,1143
29565,81 years 01 month,973


### Feature Engineering - 'storey_range'

In [121]:
def convert_storey_range(storey_range):
    """
    Convert storey range to the numerical average.
    Args:
        storey_range in (str)

    Returns: 
        float

    Example:
        convert_storey_range('07 TO 09') -> 8.0 
    """

    low, high = storey_range.split(' TO ')
    average = (int(low) + int(high)) / 2
    return average

In [122]:
def test_convert_storey_range():
    """Test cases for convert_storey_range function"""
    
    # Basic test case from the example
    assert convert_storey_range('07 TO 09') == 8.0
    
    # Test with single digits
    assert convert_storey_range('1 TO 3') == 2.0
    assert convert_storey_range('5 TO 7') == 6.0
    
    # Test with same storey (no range)
    assert convert_storey_range('05 TO 05') == 5.0
    assert convert_storey_range('10 TO 10') == 10.0
    
    # Test with larger ranges
    assert convert_storey_range('01 TO 05') == 3.0
    assert convert_storey_range('10 TO 20') == 15.0
    
    # Test with double digits
    assert convert_storey_range('12 TO 16') == 14.0
    assert convert_storey_range('25 TO 35') == 30.0
    
    # Test with leading zeros
    assert convert_storey_range('01 TO 03') == 2.0
    assert convert_storey_range('08 TO 12') == 10.0
    
    # Test odd ranges (result should be .5)
    assert convert_storey_range('1 TO 2') == 1.5
    assert convert_storey_range('10 TO 11') == 10.5
    
    # Test with higher floors
    assert convert_storey_range('50 TO 60') == 55.0
    assert convert_storey_range('99 TO 101') == 100.0

    print("All test cases passed!")


test_convert_storey_range()


All test cases passed!


In [123]:
def test_convert_storey_range_edge_cases():
    """Test edge cases and potential error scenarios"""
    
    # Test ground floor scenarios
    assert convert_storey_range('0 TO 2') == 1.0
    assert convert_storey_range('00 TO 01') == 0.5
    
    # Test with mixed formatting
    assert convert_storey_range('5 TO 15') == 10.0
    assert convert_storey_range('02 TO 8') == 5.0

    print("All edge case tests passed!")

test_convert_storey_range_edge_cases()

All edge case tests passed!


In [124]:
import pytest
def test_convert_storey_range_errors():
    """Test error handling scenarios"""
    
    # Test invalid format (should raise ValueError)
    with pytest.raises(ValueError):
        convert_storey_range('invalid format')
    
    with pytest.raises(ValueError):
        convert_storey_range('5-7')  # Wrong separator
    
    with pytest.raises(ValueError):
        convert_storey_range('5 TO')  # Missing high value
    
    with pytest.raises(ValueError):
        convert_storey_range('TO 7')  # Missing low value
    
    # Test non-numeric values
    with pytest.raises(ValueError):
        convert_storey_range('A TO B')
    
    with pytest.raises(ValueError):
        convert_storey_range('1 TO B')

    print("All error handling tests passed!")

test_convert_storey_range_errors()

All error handling tests passed!


In [125]:
# Alternative test format using unittest if you prefer
import unittest

class TestConvertStoreyRange(unittest.TestCase):
    
    def test_basic_functionality(self):
        """Test basic functionality"""
        self.assertEqual(convert_storey_range('07 TO 09'), 8.0)
        self.assertEqual(convert_storey_range('1 TO 3'), 2.0)
        self.assertEqual(convert_storey_range('10 TO 20'), 15.0)
        print("Basic functionality test passed!")
    
    def test_same_storey(self):
        """Test when low and high are the same"""
        self.assertEqual(convert_storey_range('05 TO 05'), 5.0)
        self.assertEqual(convert_storey_range('10 TO 10'), 10.0)
        print("Same storey test passed!")
    
    def test_decimal_results(self):
        """Test cases that result in decimal values"""
        self.assertEqual(convert_storey_range('1 TO 2'), 1.5)
        self.assertEqual(convert_storey_range('10 TO 11'), 10.5)
        print("Decimal results test passed!")
    
    def test_invalid_input(self):
        """Test invalid input handling"""
        with self.assertRaises(ValueError):
            convert_storey_range('invalid')
        with self.assertRaises(ValueError):
            convert_storey_range('A TO B')
        print("Invalid input test passed!")


unit_test = TestConvertStoreyRange()
unit_test.test_basic_functionality()
unit_test.test_same_storey()
unit_test.test_decimal_results()
unit_test.test_invalid_input()

Basic functionality test passed!
Same storey test passed!
Decimal results test passed!
Invalid input test passed!


In [126]:
# convert storey range to number middle value
df['storey_range'] = df.storey_range.apply(convert_storey_range)

In [127]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,transaction_year,transaction_month,remaining_lease_by_months
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,11.0,44.0,Improved,1979,61 years 04 months,232000.0,2017,1,736
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,2.0,67.0,New Generation,1978,60 years 07 months,250000.0,2017,1,727
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,2.0,67.0,New Generation,1980,62 years 05 months,262000.0,2017,1,749
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,5.0,68.0,New Generation,1980,62 years 01 month,265000.0,2017,1,745
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,2.0,67.0,New Generation,1980,62 years 05 months,265000.0,2017,1,749


### Dropping Irrelevant Columns

In [128]:
# Dropping irrelevant columns for machine learning preparation
# Keep month for temporal sorting and split, will remove later
irrelevant_columns = ['block', 'street_name', 'remaining_lease', 'lease_commence_date']
df.drop(columns = irrelevant_columns, inplace=True)

In [129]:
df.head()

,month,town,flat_type,storey_range,floor_area_sqm,flat_model,resale_price,transaction_year,transaction_month,remaining_lease_by_months
0,2017-01-01,ANG MO KIO,2 ROOM,11.0,44.0,Improved,232000.0,2017,1,736
1,2017-01-01,ANG MO KIO,3 ROOM,2.0,67.0,New Generation,250000.0,2017,1,727
2,2017-01-01,ANG MO KIO,3 ROOM,2.0,67.0,New Generation,262000.0,2017,1,749
3,2017-01-01,ANG MO KIO,3 ROOM,5.0,68.0,New Generation,265000.0,2017,1,745
4,2017-01-01,ANG MO KIO,3 ROOM,2.0,67.0,New Generation,265000.0,2017,1,749


In [130]:
df.columns

Index(['month', 'town', 'flat_type', 'storey_range', 'floor_area_sqm',
       'flat_model', 'resale_price', 'transaction_year', 'transaction_month',
       'remaining_lease_by_months'],
      dtype='object')

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 210782 entries, 0 to 211085
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   month                      210782 non-null  datetime64[ns]
 1   town                       210782 non-null  object        
 2   flat_type                  210782 non-null  object        
 3   storey_range               210782 non-null  float64       
 4   floor_area_sqm             210782 non-null  float64       
 5   flat_model                 210782 non-null  object        
 6   resale_price               210782 non-null  float64       
 7   transaction_year           210782 non-null  int32         
 8   transaction_month          210782 non-null  int32         
 9   remaining_lease_by_months  210782 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int32(2), int64(1), object(3)
memory usage: 16.1+ MB


## Data Split

### Temporal Split

In [132]:
X = df.set_index('month').sort_index().copy() 
y = X['resale_price']
X = X.drop(columns=['resale_price'])

In [133]:
X.head()

,town,flat_type,storey_range,floor_area_sqm,flat_model,transaction_year,transaction_month,remaining_lease_by_months
month,,,,,,,,
2017-01-01,ANG MO KIO,2 ROOM,11.0,44.0,Improved,2017,1,736
2017-01-01,SEMBAWANG,4 ROOM,2.0,100.0,Model A,2017,1,981
2017-01-01,SEMBAWANG,4 ROOM,5.0,100.0,Model A,2017,1,976
2017-01-01,SEMBAWANG,4 ROOM,2.0,86.0,Model A2,2017,1,990
2017-01-01,QUEENSTOWN,5 ROOM,20.0,110.0,Improved,2017,1,1067


In [134]:
y.head()

month
2017-01-01    232000.0
2017-01-01    342000.0
2017-01-01    335000.0
2017-01-01    305000.0
2017-01-01    860000.0
Name: resale_price, dtype: float64

In [135]:
print("X_temporal shape:", X.shape)
print("y_temporal shape:", y.shape)

X_temporal shape: (210782, 8)
y_temporal shape: (210782,)


In [136]:
# find split date for training and validation
split_date_train = df['month'].quantile(0.6)
split_date_train

Timestamp('2022-05-01 00:00:00')

In [137]:
split_date_validation = df['month'].quantile(0.8) 
split_date_validation

Timestamp('2023-12-01 00:00:00')

In [138]:
# Split the data into training and testing sets in a temporal manner
X_train = X[X.index < split_date_train]
y_train = y[y.index < split_date_train]
X_validation = X[(X.index >= split_date_train) & (X.index < split_date_validation)]
y_validation = y[(y.index >= split_date_train) & (y.index < split_date_validation)]
X_test = X[X.index >= split_date_validation]
y_test = y[y.index >= split_date_validation]


In [139]:
# Display the shapes of the splits to verify
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_validation.shape, y_validation.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Training set shape: (125284, 8) (125284,)
Validation set shape: (41584, 8) (41584,)
Test set shape: (43914, 8) (43914,)


## Preprocessing Setup  

In [140]:
X_train.head()

,town,flat_type,storey_range,floor_area_sqm,flat_model,transaction_year,transaction_month,remaining_lease_by_months
month,,,,,,,,
2017-01-01,ANG MO KIO,2 ROOM,11.0,44.0,Improved,2017,1,736
2017-01-01,SEMBAWANG,4 ROOM,2.0,100.0,Model A,2017,1,981
2017-01-01,SEMBAWANG,4 ROOM,5.0,100.0,Model A,2017,1,976
2017-01-01,SEMBAWANG,4 ROOM,2.0,86.0,Model A2,2017,1,990
2017-01-01,QUEENSTOWN,5 ROOM,20.0,110.0,Improved,2017,1,1067


In [141]:
numerical_features = X_train.select_dtypes(include='number').columns.tolist()
categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()
temporal_columns = X_train.select_dtypes(include=['datetime']).columns.tolist()


In [142]:
numerical_features

['storey_range',
 'floor_area_sqm',
 'transaction_year',
 'transaction_month',
 'remaining_lease_by_months']

In [143]:
categorical_columns

['town', 'flat_type', 'flat_model']

In [144]:
temporal_columns

[]

### Numerical Preprocessing

In [145]:
# Select columns that are numerical for feature scaling preparation
numerical_features = ['floor_area_sqm', 'remaining_lease_by_months', 'transaction_year'] 

degree = 1  # Degree of polynomial features, can be adjusted
# Create a numerical transformer pipeline
numerical_transformer = Pipeline(steps=[
    ('polynomial_features', PolynomialFeatures(degree=degree)),  # Placeholder for polynomial features
    ('scaler', StandardScaler())
])

### Nominal Preprocessing

In [146]:
# Select columns that need to be one-hot encoded
nominal_features = ['transaction_month', 'town', 'flat_model', 'flat_type']

# Setting pipeline for one-hot encoding
nominal_transformer = Pipeline(steps=[
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
])


### Passthrough Preprocessing

In [147]:
# Select the columns that do not required further processing 
passthrough_features = ['storey_range']

## Preprocessor

In [148]:
# tree base preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('nom', nominal_transformer, nominal_features),
        ('pass', 'passthrough', passthrough_features) 
    ],
    remainder='passthrough',
    n_jobs=-1
    )

In [149]:
# print numerical, nominal and passthrough features
print("Numerical features:", numerical_features)
print("Nominal features:", nominal_features)    
print("Passthrough features:", passthrough_features)

Numerical features: ['floor_area_sqm', 'remaining_lease_by_months', 'transaction_year']
Nominal features: ['transaction_month', 'town', 'flat_model', 'flat_type']
Passthrough features: ['storey_range']


In [150]:
preprocessor

ColumnTransformer(n_jobs=-1, remainder='passthrough',
                  transformers=[('num', 'passthrough',
                                 ['floor_area_sqm', 'remaining_lease_by_months',
                                  'transaction_year']),
                                ('nom',
                                 Pipeline(steps=[('one_hot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['transaction_month', 'town', 'flat_model',
                                  'flat_type']),
                                ('pass', 'passthrough', ['storey_range'])])

## Model Development - Tree-based Model

### Decision Tree Based Baseline Model

In [151]:
# Decision Tree Regressor

dt_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=42))
])  

In [152]:
dt_pipeline.fit(X_train, y_train) 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('num', 'passthrough',
                                                  ['floor_area_sqm',
                                                   'remaining_lease_by_months',
                                                   'transaction_year']),
                                                 ('nom',
                                                  Pipeline(steps=[('one_hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['transaction_month', 'town',
                                                   'flat_model', 'flat_type']),
                                                 ('pass', 'passthrough',
                                                  ['storey_range'])])),
                ('regressor', DecisionTreeRegressor(random_state=42))])

In [153]:
def predict_and_evaluate_validation(pipeline, X_validation: pd.DataFrame, y_validation):
    """    Predict and evaluate the model using the provided pipeline.
    Args:
        pipeline (Pipeline): The machine learning pipeline to use for prediction.
        X_validation (pd.DataFrame): The feature set for prediction.
        y_validation (pd.Series): The true target values for evaluation.
    Returns:
        dict: A dictionary containing the evaluation metrics.
        prints the metrics to the console.
    """

    # Predict on the validation set with given pipeline
    y_prediction = pipeline.predict(X_validation)  
    
    # Calculate regression metrics for validation set
    validation_mae = mean_absolute_error(y_validation, y_prediction)
    validation_mse = mean_squared_error(y_validation, y_prediction)
    validation_rmse = root_mean_squared_error(y_validation, y_prediction)
    validation_r2 = r2_score(y_validation, y_prediction)  

    model_name = pipeline.named_steps['regressor'].__class__.__name__

    # Display the metrics for Decision Tree Regressor
    print(f"{model_name} Metrics:")
    print(f"{model_name} Validation MAE: {validation_mae}")
    print(f"{model_name} Validation MSE: {validation_mse}")
    print(f"{model_name} Validation RMSE: {validation_rmse}")
    print(f"{model_name} Validation R²: {validation_r2}")

    return {'MAE':validation_mae, 'MSE':validation_mse, 'RMSE':validation_rmse, 'R2':validation_r2}

In [154]:
validation_dt = predict_and_evaluate_validation(dt_pipeline, X_validation, y_validation)

DecisionTreeRegressor Metrics:
DecisionTreeRegressor Validation MAE: 56238.6459159773
DecisionTreeRegressor Validation MSE: 5833637012.623298
DecisionTreeRegressor Validation RMSE: 76378.24960434284
DecisionTreeRegressor Validation R²: 0.8046663920097824


In [155]:
def importance_features(pipeline):
    """Display the feature importance from the pipeline."""
    # Get feature importance from the regressor step
    feature_importance = pipeline.named_steps['regressor'].feature_importances_
    
    # Get feature names from the preprocessor step
    feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
    
    # Create a DataFrame for feature importance
    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
    
    # Sort by importance
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
    
    # Display the top 20 features
    print("Top 20 Features by Importance:")
    print(feature_importance_df.head(20))
    return feature_importance_df

In [156]:
dt_feature_importance_df = importance_features(dt_pipeline)

Top 20 Features by Importance:
                           Feature  Importance
0              num__floor_area_sqm    0.439813
68              pass__storey_range    0.111728
1   num__remaining_lease_by_months    0.091611
2            num__transaction_year    0.040151
44            nom__flat_model_DBSS    0.036381
19           nom__town_BUKIT MERAH    0.032162
33            nom__town_QUEENSTOWN    0.025267
17                nom__town_BISHAN    0.018798
22          nom__town_CENTRAL AREA    0.017703
35              nom__town_SENGKANG    0.014236
39             nom__town_WOODLANDS    0.013712
29       nom__town_KALLANG/WHAMPOA    0.012516
38             nom__town_TOA PAYOH    0.009863
23         nom__town_CHOA CHU KANG    0.009767
25               nom__town_GEYLANG    0.009365
20         nom__town_BUKIT PANJANG    0.007518
24              nom__town_CLEMENTI    0.007125
21           nom__town_BUKIT TIMAH    0.006770
15            nom__town_ANG MO KIO    0.006131
48         nom__flat_model_Mo

**Using basic decision tree model without any parameter tuning, our error reduced and performance improve is much better than Linear Regression with transformed target. We have a MAE of $56K and R-squared of 0.8046. We will use decision tree with default parameters as our baseline model.**
 

### Random Forest

In [157]:
# Random Forest Regressor
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))
])
rf_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('num', 'passthrough',
                                                  ['floor_area_sqm',
                                                   'remaining_lease_by_months',
                                                   'transaction_year']),
                                                 ('nom',
                                                  Pipeline(steps=[('one_hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['transaction_month', 'town',
                                                   'flat_model', 'flat_type']),
                                                 ('pass', 'passthrough',
                                                  ['storey_range'])])),
                ('regressor',
                 RandomForestRegressor(n_jobs=-1, random_state=42))])

In [158]:
validation_rf = predict_and_evaluate_validation(rf_pipeline, X_validation, y_validation)

RandomForestRegressor Metrics:
RandomForestRegressor Validation MAE: 49401.2283803187
RandomForestRegressor Validation MSE: 4384385916.313756
RandomForestRegressor Validation RMSE: 66214.69562199735
RandomForestRegressor Validation R²: 0.8531931421166803


In [159]:
rf_feature_importance_df = importance_features(rf_pipeline) 

Top 20 Features by Importance:
                           Feature  Importance
0              num__floor_area_sqm    0.433222
68              pass__storey_range    0.116265
1   num__remaining_lease_by_months    0.093843
2            num__transaction_year    0.039055
19           nom__town_BUKIT MERAH    0.033764
44            nom__flat_model_DBSS    0.031180
33            nom__town_QUEENSTOWN    0.026016
17                nom__town_BISHAN    0.019321
22          nom__town_CENTRAL AREA    0.014584
64           nom__flat_type_4 ROOM    0.013666
39             nom__town_WOODLANDS    0.013319
29       nom__town_KALLANG/WHAMPOA    0.012761
35              nom__town_SENGKANG    0.012267
25               nom__town_GEYLANG    0.010167
38             nom__town_TOA PAYOH    0.009927
23         nom__town_CHOA CHU KANG    0.008929
24              nom__town_CLEMENTI    0.007452
20         nom__town_BUKIT PANJANG    0.007255
21           nom__town_BUKIT TIMAH    0.006708
15            nom__town_ANG M

### XGBoost

In [160]:
# XGBoost Regressor
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(random_state=42, n_jobs=-1, verbosity=0))
])
xgb_pipeline.fit(X_train, y_train)  

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('num', 'passthrough',
                                                  ['floor_area_sqm',
                                                   'remaining_lease_by_months',
                                                   'transaction_year']),
                                                 ('nom',
                                                  Pipeline(steps=[('one_hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['transaction_month', 'town',
                                                   'flat_model', 'flat_type']),
                                                 ('pass', 'passthrough',
                                                  ['storey_ra...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=-1,
                              num_parallel_tree=None, ...))])

In [161]:
validation_xgb = predict_and_evaluate_validation(xgb_pipeline, X_validation, y_validation)

XGBRegressor Metrics:
XGBRegressor Validation MAE: 44359.52789663693
XGBRegressor Validation MSE: 3547355844.5275393
XGBRegressor Validation RMSE: 59559.68304589556
XGBRegressor Validation R²: 0.8812202722868498


In [162]:
# feature importance df
xgb_feature_importance_df = importance_features(xgb_pipeline) 

Top 20 Features by Importance:
                      Feature  Importance
62      nom__flat_type_2 ROOM    0.164108
64      nom__flat_type_4 ROOM    0.094579
19      nom__town_BUKIT MERAH    0.071786
33       nom__town_QUEENSTOWN    0.067525
44       nom__flat_model_DBSS    0.057087
17           nom__town_BISHAN    0.054181
22     nom__town_CENTRAL AREA    0.045579
23    nom__town_CHOA CHU KANG    0.035215
39        nom__town_WOODLANDS    0.031018
29  nom__town_KALLANG/WHAMPOA    0.029805
0         num__floor_area_sqm    0.026668
21      nom__town_BUKIT TIMAH    0.025575
35         nom__town_SENGKANG    0.024886
38        nom__town_TOA PAYOH    0.020846
30    nom__town_MARINE PARADE    0.020068
34        nom__town_SEMBAWANG    0.019234
25          nom__town_GEYLANG    0.016044
24         nom__town_CLEMENTI    0.015721
28      nom__town_JURONG WEST    0.015683
68         pass__storey_range    0.015107


### LightGBM

In [163]:
# LightGBM Regressor
lgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(random_state=42, n_jobs=-1, verbosity=-1))
])
lgb_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('num', 'passthrough',
                                                  ['floor_area_sqm',
                                                   'remaining_lease_by_months',
                                                   'transaction_year']),
                                                 ('nom',
                                                  Pipeline(steps=[('one_hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['transaction_month', 'town',
                                                   'flat_model', 'flat_type']),
                                                 ('pass', 'passthrough',
                                                  ['storey_range'])])),
                ('regressor',
                 LGBMRegressor(n_jobs=-1, random_state=42, verbosity=-1))])

In [164]:
validation_lgb = predict_and_evaluate_validation(lgb_pipeline, X_validation, y_validation)

LGBMRegressor Metrics:
LGBMRegressor Validation MAE: 47256.30873600356
LGBMRegressor Validation MSE: 4379605909.772255
LGBMRegressor Validation RMSE: 66178.59102286973
LGBMRegressor Validation R²: 0.8533531959427836


/opt/miniconda3/envs/hdbenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [165]:
# feature importance df
lgb_feature_importance_df = importance_features(lgb_pipeline) 

Top 20 Features by Importance:
                           Feature  Importance
0              num__floor_area_sqm         500
1   num__remaining_lease_by_months         453
2            num__transaction_year         204
68              pass__storey_range         176
19           nom__town_BUKIT MERAH          97
33            nom__town_QUEENSTOWN          93
17                nom__town_BISHAN          70
29       nom__town_KALLANG/WHAMPOA          66
25               nom__town_GEYLANG          63
38             nom__town_TOA PAYOH          61
24              nom__town_CLEMENTI          58
22          nom__town_CENTRAL AREA          57
39             nom__town_WOODLANDS          56
28           nom__town_JURONG WEST          56
15            nom__town_ANG MO KIO          54
23         nom__town_CHOA CHU KANG          49
21           nom__town_BUKIT TIMAH          48
16                 nom__town_BEDOK          46
30         nom__town_MARINE PARADE          45
35              nom__town_SEN

### CatGBM

In [166]:
# CatGBM Regressor
cgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', CatBoostRegressor(random_state=42, verbose=0))
])
cgb_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('num', 'passthrough',
                                                  ['floor_area_sqm',
                                                   'remaining_lease_by_months',
                                                   'transaction_year']),
                                                 ('nom',
                                                  Pipeline(steps=[('one_hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['transaction_month', 'town',
                                                   'flat_model', 'flat_type']),
                                                 ('pass', 'passthrough',
                                                  ['storey_range'])])),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x14b6b9d90>)])

In [167]:
validation_cgb = predict_and_evaluate_validation(cgb_pipeline, X_validation, y_validation)

CatBoostRegressor Metrics:
CatBoostRegressor Validation MAE: 44289.47124331475
CatBoostRegressor Validation MSE: 3439507638.844653
CatBoostRegressor Validation RMSE: 58647.31570024883
CatBoostRegressor Validation R²: 0.8848314635703878


In [168]:
# feature importance df
cgb_feature_importance_df = importance_features(cgb_pipeline) 

Top 20 Features by Importance:
                           Feature  Importance
0              num__floor_area_sqm   39.395083
1   num__remaining_lease_by_months   10.919752
68              pass__storey_range    7.508472
64           nom__flat_type_4 ROOM    5.353658
19           nom__town_BUKIT MERAH    4.543736
2            num__transaction_year    4.261036
33            nom__town_QUEENSTOWN    3.502339
44            nom__flat_model_DBSS    2.377892
17                nom__town_BISHAN    2.246863
22          nom__town_CENTRAL AREA    1.736499
38             nom__town_TOA PAYOH    1.663576
29       nom__town_KALLANG/WHAMPOA    1.653027
39             nom__town_WOODLANDS    1.492214
25               nom__town_GEYLANG    1.221238
23         nom__town_CHOA CHU KANG    1.077376
28           nom__town_JURONG WEST    1.031848
35              nom__town_SENGKANG    0.931542
24              nom__town_CLEMENTI    0.884924
40                nom__town_YISHUN    0.706105
15            nom__town_ANG M

In [169]:
# add ing metrics to dictionary for each model
metrics_dt = {
    "Model": "Decision Tree",
    "MAE":  validation_dt['MAE'],
    "RMSE": validation_dt['RMSE'],
    "R2":   validation_dt['R2'],
}

metrics_lgb = {
    "Model": "LightGBM",
    "MAE":  validation_lgb['MAE'],
    "RMSE": validation_lgb['RMSE'],
    "R2":   validation_lgb['R2'],
}

metrics_rf = {
    "Model": "Random Forest",
    "MAE":  validation_rf['MAE'],
    "RMSE": validation_rf['RMSE'],
    "R2":   validation_rf['R2'],
}

metrics_xgb = {
    "Model": "XGBoost",
    "MAE":  validation_xgb['MAE'],
    "RMSE": validation_xgb['RMSE'],
    "R2":   validation_xgb['R2'],
}

metrics_cgb = {
    "Model": "CatGBoost",
    "MAE":  validation_cgb['MAE'],
    "RMSE": validation_cgb['RMSE'],
    "R2":   validation_cgb['R2'],
}


all_results = [metrics_dt, metrics_rf, metrics_xgb, metrics_lgb, metrics_cgb]

results_df = (
    pd.DataFrame(all_results)
      .set_index("Model")
      .round(4)          # nice, tidy formatting
)

In [170]:
results_df = results_df.sort_values('MAE', ascending=True)

In [171]:
display(results_df)

,MAE,RMSE,R2
Model,,,
CatGBoost,44289.4712,58647.3157,0.8848
XGBoost,44359.5279,59559.6830,0.8812
LightGBM,47256.3087,66178.5910,0.8534
Random Forest,49401.2284,66214.6956,0.8532
Decision Tree,56238.6459,76378.2496,0.8047


In [172]:
# Saved for final sanity check
dt_default_model = dt_pipeline
rf_default_model = rf_pipeline

**Without any parameters tuning, CatGBoost and XGBoost both shows the most promising results, it is followed by Random Forest and LightGBM. We will use the 4 models for further hyperparameter tuning. Decision tree without tuning will be the base line model for sanity check.** 

## Performance Metrics

In [173]:
display(results_df)

,MAE,RMSE,R2
Model,,,
CatGBoost,44289.4712,58647.3157,0.8848
XGBoost,44359.5279,59559.6830,0.8812
LightGBM,47256.3087,66178.5910,0.8534
Random Forest,49401.2284,66214.6956,0.8532
Decision Tree,56238.6459,76378.2496,0.8047


**MSE is too difficult to interpret and thus we have dropped it from the analysis. We will be using MAE, RMSE and R-squared. We should look at all the different performance metrics MAE, RMSE and R2 because different performance metric present different information. MAE measure the magnitude of errors but it is less sensitive to outliers. RMSE penalized large errors and helps to identify if you have large outliers. R2 tell us how much of the variation in the target are explainable by our model. However, it cannot tell us if the predictions are bias.**

**Our analysis above shows that RMSE are larger than MAE indicating that there are outliers that RMSE amplified. We will not use RMSE as we do not want to penalized the squared error since the outliers are the high end housing market that our property firm would want to served. Our R-squared are consistently high indicating that our tree-based models can explain the variation in the target better. However, the differences between difference R-squared could not help us to explain the metrics further. In conclusion, we would use MAE as our primary metrics as it is also easier to explained to the management of the property firm.**   

## Hyperparameter Tuning

**Grid Search Strategy**

We will use **MAE** as the main metric because it is most easy to be understood by the stakeholder.

We have performed fine tuning with a 3 stage parameters fine tuning starting with **Halving Randomized Search** for stage 1 using the widest search space. Then we will use **Optuna Search** to finalized the search parameters.

However, after several hours of fine tuning, our MAE did improved a few hundred dollars. This is not acceptable as the improvement is marginal compared to the resource we have put in. Thus we will be using randomized search  with ranges around the default as sanity check.

For further fine tuning, we will be using optuna with 5 cross validation folder around our searched parameters.

### Random Search CV Parameters Settings

In [ ]:
# Define Randomized SearchCV parameters with wide search space for 
# Random Forest Regressor, XGBoost Regressor, LightGBM Regressor and CatGBoost regressor
from scipy.stats import randint, loguniform, uniform

# 1) Random Forest  ------------------------------------------------------
rf_param_dist = {
    "regressor__n_estimators":      randint(200, 800),
    "regressor__max_depth":         [None] + list(range(10, 41, 10)),
    "regressor__max_features":      ["sqrt", "log2", None],
    "regressor__min_samples_split": randint(2, 11),
    "regressor__min_samples_leaf":  randint(1, 21),
    "regressor__bootstrap":         [True, False],
}

# 2) XGBoost -------------------------------------------------------------
xgb_param_dist = {
    "regressor__n_estimators":        randint(200, 1500),
    "regressor__learning_rate":       loguniform(0.01, 0.2),        # 1e‑2 – 2e‑1
    "regressor__max_depth":           randint(3, 11),
    "regressor__min_child_weight":    loguniform(0.5, 10),
    "regressor__subsample":           uniform(0.5, 0.5),            # 0.5 – 1.0
    "regressor__colsample_bytree":    uniform(0.5, 0.5),
    "regressor__gamma":               uniform(0, 5),
    "regressor__reg_alpha":           loguniform(1e-3, 10),
    "regressor__reg_lambda":          loguniform(1e-2, 10),
}

# 3) LightGBM ------------------------------------------------------------
lgb_param_dist = {
    "regressor__num_leaves":          randint(20, 300),
    "regressor__max_depth":           randint(-1, 50),              # ‑1 means “no limit”
    "regressor__learning_rate":       loguniform(0.01, 0.3),
    "regressor__n_estimators":        randint(200, 1500),
    "regressor__min_child_samples":   randint(5, 100),
    "regressor__subsample":           uniform(0.5, 0.5),
    "regressor__colsample_bytree":    uniform(0.5, 0.5),
    "regressor__reg_alpha":           loguniform(1e-3, 10),
    "regressor__reg_lambda":          loguniform(1e-2, 10),
    "regressor__boosting_type":       ["gbdt", "dart", "goss"],
}

# 4) CatBoost ------------------------------------------------------------
cat_param_dist = {
    "regressor__iterations":          randint(300, 2000),
    "regressor__learning_rate":       loguniform(0.01, 0.3),
    "regressor__depth":               randint(4, 11),
    "regressor__l2_leaf_reg":         loguniform(1, 10),
    "regressor__bagging_temperature": uniform(0, 1),
    "regressor__random_strength":     uniform(0, 1),
    "regressor__border_count":        randint(32, 255),
}

In [ ]:
rf_random_search = HalvingRandomSearchCV(
    estimator=rf_pipeline,                 # rf_pipeline should set regressor__random_state=42
    param_distributions=rf_param_dist,
    resource="n_estimators",               # 🔑 use trees as the budget axis
    min_resources=200,
    max_resources=800,
    factor=4,                              # steeper pruning
    aggressive_elimination=True,
    scoring="neg_mean_absolute_error",
    cv=3,
    random_state=42,
    n_jobs=-1,
    verbose=2,
    error_score="raise",
)
rf_random_search.fit(X_train, y_train)

n_iterations: 10
n_required_iterations: 10
n_possible_iterations: 10
min_resources_: 6
max_resources_: 125284
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 20880
n_resources: 6
Fitting 3 folds for each of 20880 candidates, totalling 62640 fits
[CV 3/3] END regressor__bootstrap=True, regressor__max_depth=60, regressor__max_features=log2, regressor__min_samples_leaf=3, regressor__min_samples_split=8, regressor__n_estimators=530;, score=(train=-20834.670, test=-169169.340) total time=   1.3s
[CV 2/3] END regressor__bootstrap=True, regressor__max_depth=60, regressor__max_features=log2, regressor__min_samples_leaf=3, regressor__min_samples_split=8, regressor__n_estimators=530;, score=(train=-24241.509, test=-17516.981) total time=   1.3s
[CV 1/3] END regressor__bootstrap=True, regressor__max_depth=60, regressor__max_features=log2, regressor__min_samples_leaf=3, regressor__min_samples_split=8, regressor__n_estimators=530;, score=(train=-218368.553, test=-183000.000)

KeyboardInterrupt: 

In [ ]:
# perform randomized search for XGBoost Regressor
xgb_random_search = RandomizedSearchCV(
    xgb_pipeline,
    param_distributions=xgb_param_grid,
    n_iter=60,  # Number of iterations for random search
    scoring='neg_mean_absolute_error',   
    cv=3,  # 3-fold cross-validation
    verbose=3,
    random_state=42,
    n_jobs=-1  # Use all available cores
)
xgb_random_search.fit(X_train, y_train) 

In [ ]:
# perform randomized search for LightGBM Regressor
lgb_random_search = RandomizedSearchCV(
    lgb_pipeline,
    param_distributions=lgb_param_grid,
    n_iter=30,  # Number of iterations for random search
    scoring='neg_mean_absolute_error',       
    cv=3,  # 3-fold cross-validation
    verbose=3,
    random_state=42,
    n_jobs=-1  # Use all available cores
)
lgb_random_search.fit(X_train, y_train)

In [ ]:
# Print the best parameters and best score for Random Forest Regressor
print("Best parameters for Random Forest Regressor:", rf_random_search.best_params_)
print("Best score for Random Forest Regressor (negative MSE):", rf_random_search.best_score_)
# Print the best parameters and best score for XGBoost Regressor
print("Best parameters for XGBoost Regressor:", xgb_random_search.best_params_)
print("Best score for XGBoost Regressor (negative MSE):", xgb_random_search.best_score_)
# Print the best parameters and best score for LightGBM Regressor
print("Best parameters for LightGBM Regressor:", lgb_random_search.best_params_)
print("Best score for LightGBM Regressor (negative MSE):", lgb_random_search.best_score_)

**The difference between the best model is not that great. Will perform fine tuning and select the best few for final model evaluation test.**

### Optuna Grid Search CV

In [ ]:
def make_tight_distributions(best_params,
                             int_frac: float = 0.2,
                             float_frac: float = 0.1,
                             min_int_step: int = 1):
    """
    Given a dict of best_params_, return a dict of
    Optuna Distributions that span ±frac around each value.
    """
    tight_dists = {}
    for name, val in best_params.items():
        # only handle numeric params
        if isinstance(val, int):
            # window = max(val * int_frac, min_int_step)
            window = max(int(val * int_frac), min_int_step)
            low  = max(1, val - window)      # avoid zero or negative
            high = val + window
            # choose step = min_int_step or window itself
            step = min_int_step if min_int_step <= window else window
            tight_dists[name] = IntDistribution(low=low, high=high, step=step)

        elif isinstance(val, float):
            window = val * float_frac
            low  = max(0.0, val - window)
            high = min(1.0, val + window)    # assuming [0,1] support for fractions
            tight_dists[name] = FloatDistribution(low=low, high=high)

        else:
            # skip non-numeric (e.g. categorical) or handle separately
            continue

    return tight_dists


#### Random Forest Optuna Search CV

In [ ]:

# 1. grab your previously-found best params:
best_rf_random_search = rf_random_search.best_params_

# 2. build the “around-the-best” distributions:
rf_param_distributions = make_tight_distributions(best_rf_random_search,
                                               int_frac=0.2,     # ±20%
                                               float_frac=0.1,   # ±10%
                                               min_int_step=1)   # at least step=1


In [ ]:
rf_param_distributions

In [ ]:
# 3. Optuna quick search
optuna_search = OptunaSearchCV(
    estimator=rf_pipeline,
    param_distributions=rf_param_distributions,
    cv=5,             
    n_trials=2,      
    scoring='neg_mean_absolute_error',
    random_state=42,
    n_jobs=-1,
    verbose=3,
)

# 4. Run the search
optuna_search.fit(X_train, y_train)


In [ ]:
# 5. Inspect results
print("Best MAE  =", optuna_search.best_score_)
print("Best params:")
for k, v in optuna_search.best_params_.items():
    print(f"  • {k} = {v}")

# 6. Your final model
final_rf_model = optuna_search.best_estimator_
final_rf_model_params = optuna_search.best_params_
final_rf_model_scores = optuna_search.best_score_

In [ ]:
val_final_rf_model_pred = final_rf_model.predict(X_val)

# 7. Calculate regression metrics for validation set with Random Forest Regressor
final_val_mae_rf = mean_absolute_error(y_val, val_final_rf_model_pred)
final_val_mse_rf = mean_squared_error(y_val, val_final_rf_model_pred)
final_val_rmse_rf = root_mean_squared_error(y_val, val_final_rf_model_pred)
final_val_r2_rf = r2_score(y_val, val_final_rf_model_pred)

# 8. Display the metrics for Random Forest Regressor
print("Random Forest Regressor Metrics:")
print(f"Random Forest Validation MAE: {final_val_mae_rf}")
print(f"Random Forest Validation MSE: {final_val_mse_rf}")
print(f"Random Forest Validation RMSE: {final_val_rmse_rf}")
print(f"Random Forest Validation R²: {final_val_r2_rf}")

#### XGBoost Optuna Search CV

In [ ]:
# 1. grab your previously-found best params:
best_xgb_random_search_param = xgb_random_search.best_params_

# 2. build the “around-the-best” distributions:
xgb_param_distributions = make_tight_distributions(best_xgb_random_search_param,
                                               int_frac=0.2,     # ±20%
                                               float_frac=0.1,   # ±10%
                                               min_int_step=1)   # at least step=1


In [ ]:
xgb_param_distributions

In [ ]:
# 3. Optuna quick search
optuna_search = OptunaSearchCV(
    estimator=xgb_pipeline,
    param_distributions=xgb_param_distributions,
    cv=5,             
    n_trials=50,      
    scoring='neg_mean_absolute_error',
    random_state=42,
    n_jobs=-1,
    verbose=3,
)

# 4. Run the search
optuna_search.fit(X_train, y_train)

In [ ]:
# 5. Inspect results
print("Best MAE  =", optuna_search.best_score_)
print("Best params:")
for k, v in optuna_search.best_params_.items():
    print(f"  • {k} = {v}")

# 6. Your final model
final_xgb_model = optuna_search.best_estimator_
final_xgb_model_params = optuna_search.best_params_
final_xgb_model_scores = optuna_search.best_score_

In [ ]:
val_final_xgb_model = final_xgb_model.predict(X_val)

# 7. Calculate regression metrics for validation set with Random Forest Regressor
final_val_mae_xgb = mean_absolute_error(y_val, val_final_xgb_model)
final_val_mse_xgb = mean_squared_error(y_val, val_final_xgb_model)
final_val_rmse_xgb = root_mean_squared_error(y_val, val_final_xgb_model)
final_val_r2_xgb = r2_score(y_val, val_final_xgb_model)

# 8. Display the metrics for Random Forest Regressor
print("Random Forest Regressor Metrics:")
print(f"Random Forest Validation MAE: {final_val_mae_xgb}")
print(f"Random Forest Validation MSE: {final_val_mse_xgb}")
print(f"Random Forest Validation RMSE: {final_val_rmse_xgb}")
print(f"Random Forest Validation R²: {final_val_r2_xgb}")

#### Light GBM Optuna Search CV

In [ ]:
# 1. grab your previously-found best params:
best_lgb_random_search_param = lgb_random_search.best_params_

# 2. build the “around-the-best” distributions:
lgb_param_distributions = make_tight_distributions(best_lgb_random_search_param,
                                               int_frac=0.2,     # ±20%
                                               float_frac=0.1,   # ±10%
                                               min_int_step=1)   # at least step=1

In [ ]:
lgb_param_distributions

In [ ]:
# 3. Optuna quick search
optuna_search = OptunaSearchCV(
    estimator=lgb_pipeline,
    param_distributions=lgb_param_distributions,
    cv=5,             
    n_trials=30,      
    scoring='neg_mean_absolute_error',
    random_state=42,
    n_jobs=-1,
    verbose=3,
)

# 4. Run the search
optuna_search.fit(X_train, y_train)

In [ ]:
# 5. Inspect results
print("Best MAE  =", optuna_search.best_score_)
print("Best params:")
for k, v in optuna_search.best_params_.items():
    print(f"  • {k} = {v!r}")

# 6. Your final model
final_lgb_model = optuna_search.best_estimator_
final_lgb_model_params = optuna_search.best_params_
final_lgb_model_scores = optuna_search.best_score_

In [ ]:
val_final_lgb_model = final_lgb_model.predict(X_val)

# 7. Calculate regression metrics for validation set with Random Forest Regressor
final_val_mae_lgb = mean_absolute_error(y_val, val_final_lgb_model)
final_val_mse_lgb = mean_squared_error(y_val, val_final_lgb_model)
final_val_rmse_lgb = root_mean_squared_error(y_val, val_final_lgb_model)
final_val_r2_lgb = r2_score(y_val, val_final_lgb_model)

# 8. Display the metrics for Random Forest Regressor
print("Random Forest Regressor Metrics:")
print(f"Random Forest Validation MAE: {final_val_mae_lgb}")
print(f"Random Forest Validation MSE: {final_val_mse_lgb}")
print(f"Random Forest Validation RMSE: {final_val_rmse_lgb}")
print(f"Random Forest Validation R²: {final_val_r2_lgb}")

#### Comparing Fine Tuned Scores

In [ ]:
# Print the best parameters and best score for Random Forest Regressor
print("Best parameters for Random Forest Regressor:", final_rf_model_params)
print("Best score for Random Forest Regressor (negative MSE):", final_rf_model_scores)
# Print the best parameters and best score for XGBoost Regressor
print("Best parameters for XGBoost Regressor:", final_xgb_model_params)
print("Best score for XGBoost Regressor (negative MSE):", final_xgb_model_scores)
# Print the best parameters and best score for LightGBM Regressor
print("Best parameters for LightGBM Regressor:", final_lgb_model_params)
print("Best score for LightGBM Regressor (negative MSE):", final_lgb_model_scores)


### Evaluation of Fine Tuned Models

**First, we apply the default decision tree as baseline for sanity check.**

In [ ]:
# Predict on the validation set with the default Decision Tree Regressor
y_val_pred_dt_default = dt_default_model.predict(X_val)
# Calculate regression metrics for validation set with the default Decision Tree Regressor
val_mae_dt_default = mean_absolute_error(y_val, y_val_pred_dt_default)
val_rmse_dt_default = root_mean_squared_error(y_val, y_val_pred_dt_default)
val_r2_dt_default = r2_score(y_val, y_val_pred_dt_default)

# Display the metrics for the default Decision Tree Regressor
print("Default Decision Tree Regressor Metrics:")
print(f"Default Decision Tree Validation MAE: {val_mae_dt_default}")
print(f"Default Decision Tree Validation RMSE: {val_rmse_dt_default}")
print(f"Default Decision Tree Validation R²: {val_r2_dt_default}")

**Next, we try Random Forest with default settings as it has a good score before fine tuning.**

In [ ]:
# Predict on the test set with the default Random Forest Regressor
y_val_pred_rf_default = rf_default_model.predict(X_val)
# Calculate regression metrics for test set with the default Random Forest Regressor
val_mae_rf_default = mean_absolute_error(y_val, y_val_pred_rf_default)
val_rmse_rf_default = root_mean_squared_error(y_val, y_val_pred_rf_default)
val_r2_rf_default = r2_score(y_val, y_val_pred_rf_default)

# Display the metrics for the default Random Forest Regressor
print("Default Random Forest Regressor Metrics:")
print(f"Default Random Forest Validation MAE: {val_mae_rf_default}")
print(f"Default Random Forest Validation RMSE: {val_rmse_rf_default}")
print(f"Default Random Forest Validation R²: {val_r2_rf_default}")

In [ ]:
# predict on the validation set with the best Random Forest Regressor
y_val_pred_rf_best = final_rf_model.predict(X_val)
# Calculate regression metrics for validation set with the best Random Forest Regressor
val_mae_rf_best = mean_absolute_error(y_val, y_val_pred_rf_best)
val_rmse_rf_best = root_mean_squared_error(y_val, y_val_pred_rf_best)
val_r2_rf_best = r2_score(y_val, y_val_pred_rf_best)    

# Display the metrics for the best Random Forest Regressor
print("Best Random Forest Regressor Metrics:")
print(f"Best Random Forest Validation MAE: {val_mae_rf_best}")
print(f"Best Random Forest Validation RMSE: {val_rmse_rf_best}")
print(f"Best Random Forest Validation R²: {val_r2_rf_best}")

In [ ]:
# predict on the validation set with the best xgboost Regressor
y_val_pred_xgb_best = final_xgb_model.predict(X_val)
# Calculate regression metrics for validation set with the best XGBoost Regressor
val_mae_xgb_best = mean_absolute_error(y_val, y_val_pred_xgb_best)
val_rmse_xgb_best = root_mean_squared_error(y_val, y_val_pred_xgb_best)
val_r2_xgb_best = r2_score(y_val, y_val_pred_xgb_best)  
# Display the metrics for the best XGBoost Regressor
print("Best XGBoost Regressor Metrics:")
print(f"Best XGBoost Validation MAE: {val_mae_xgb_best}")       
print(f"Best XGBoost Validation RMSE: {val_rmse_xgb_best}")
print(f"Best XGBoost Validation R²: {val_r2_xgb_best}") 


In [ ]:
# predict on the validation set with the best lightgbm Regressor
y_val_pred_lgb_best = final_lgb_model.predict(X_val)
# Calculate regression metrics for validation set with the best LightGBM Regressor
val_mae_lgb_best = mean_absolute_error(y_val, y_val_pred_lgb_best)
val_rmse_lgb_best = root_mean_squared_error(y_val, y_val_pred_lgb_best)
val_r2_lgb_best = r2_score(y_val, y_val_pred_lgb_best)
# Display the metrics for the best LightGBM Regressor
print("Best LightGBM Regressor Metrics:")
print(f"Best LightGBM Validation MAE: {val_mae_lgb_best}")
print(f"Best LightGBM Validation RMSE: {val_rmse_lgb_best}")
print(f"Best LightGBM Validation R²: {val_r2_lgb_best}")    


In [ ]:
# add ing metrics to dictionary for each model
metrics_rf_default = {
    "Model": "Random Forest (def)",
    "MAE":  val_mae_rf_default,
    "RMSE": val_rmse_rf_default,
    "R2":   val_r2_rf_default,
}

metrics_rf = {
    "Model": "Random Forest",
    "MAE":  val_mae_rf_best,
    "RMSE": val_rmse_rf_best,
    "R2":   val_r2_rf_best,
}

metrics_xgb = {
    "Model": "XGBoost",
    "MAE":  val_mae_xgb_best,
    "RMSE": val_rmse_xgb_best,
    "R2":   val_r2_xgb_best,
}

metrics_lgb = {
    "Model": "LightGBM",
    "MAE":  val_mae_lgb_best,
    "RMSE": val_rmse_lgb_best,
    "R2":   val_r2_lgb_best,
}

all_results = [metrics_rf_default, metrics_rf, metrics_xgb, metrics_lgb]

results_df = (
    pd.DataFrame(all_results)
      .set_index("Model")
      .round(4)          # nice, tidy formatting
)

In [ ]:
display(results_df)

**Our best model is XGBoost with MAE of $23,338 margin or errors.**

**All MAE are very close. In our experience, model with the best validation score may not do well in the test. Therefore, we will apply all the 4 models into the test set. We will deploy model with the best score.**



## Final Model Evaluation

In [ ]:
# Predict on the test set with the default Random Forest Regressor
y_test_pred_rf_default = rf_default_model.predict(X_test)
# Calculate regression metrics for test set with the default Random Forest Regressor
test_mae_rf_default = mean_absolute_error(y_test, y_test_pred_rf_default)
test_rmse_rf_default = root_mean_squared_error(y_test, y_test_pred_rf_default)
test_r2_rf_default = r2_score(y_test, y_test_pred_rf_default)

# Display the metrics for the default Random Forest Regressor
print("Default Random Forest Regressor Metrics:")
print(f"Default Random Forest Test MAE: {test_mae_rf_default}")
print(f"Default Random Forest Test RMSE: {test_rmse_rf_default}")
print(f"Default Random Forest Test R²: {test_r2_rf_default}")

In [ ]:
# Predict on the test set with the best Random Forest Regressor
y_test_pred_rf_best = final_rf_model.predict(X_test)
# Calculate regression metrics for test set with the best Random Forest Regressor
test_mae_rf_best = mean_absolute_error(y_test, y_test_pred_rf_best)
test_rmse_rf_best = root_mean_squared_error(y_test, y_test_pred_rf_best)
test_r2_rf_best = r2_score(y_test, y_test_pred_rf_best)

# Display the metrics for the best Random Forest Regressor
print("Best Random Forest Regressor Metrics:")
print(f"Best Random Forest Test MAE: {test_mae_rf_best}")
print(f"Best Random Forest Test RMSE: {test_rmse_rf_best}")
print(f"Best Random Forest Test R²: {test_r2_rf_best}")

In [ ]:
# Predict on the test set with the best XGBoost Regressor
y_test_pred_xgb_best = final_xgb_model.predict(X_test)
# Calculate regression metrics for test set with the best XGBoost Regressor
test_mae_xgb_best = mean_absolute_error(y_test, y_test_pred_xgb_best)
test_rmse_xgb_best = root_mean_squared_error(y_test, y_test_pred_xgb_best)
test_r2_xgb_best = r2_score(y_test, y_test_pred_xgb_best)

# Display the metrics for the best XGBoost Regressor
print("Best XGBoost Regressor Metrics:")
print(f"Best XGBoost Regressor Test MAE: {test_mae_xgb_best}")
print(f"Best XGBoost Regressor Test RMSE: {test_rmse_xgb_best}")
print(f"Best XGBoost Regressor Test R²: {test_r2_xgb_best}")

In [ ]:
# Predict on the test set with the best LightGBM Regressor
y_test_pred_lgb_best = final_lgb_model.predict(X_test)
# Calculate regression metrics for test set with the best LightGBM Regressor
test_mae_lgb_best = mean_absolute_error(y_test, y_test_pred_lgb_best)
test_rmse_lgb_best = root_mean_squared_error(y_test, y_test_pred_lgb_best)
test_r2_lgb_best = r2_score(y_test, y_test_pred_lgb_best)

# Display the metrics for the best LightGBM Regressor
print("Best LightGBM Regressor Metrics:")
print(f"Best LightGBM Regressor Test MAE: {test_mae_lgb_best}")
print(f"Best LightGBM Regressor Test RMSE: {test_rmse_lgb_best}")
print(f"Best LightGBM Regressor Test R²: {test_r2_lgb_best}")

**The best model is still XGBoost. MAE result is similar to validation test. This confirms that our test set is representative of the validation datasets.**

### Final Model Application

In [ ]:
# Best model from Optuna Search
best_model = final_xgb_model

# Predict on the test set with Ridge Regression
y_test_pred_best_model = best_model.predict(X_test)

# Calculate regression metrics for the test set for Ridge
test_mae_best = mean_absolute_error(y_test, y_test_pred_best_model)
test_rmse_best = root_mean_squared_error(y_test, y_test_pred_best_model)
test_r2_best = r2_score(y_test, y_test_pred_best_model)

print("Best Ridge Regression Model, Final Test Metrics:")
print(f"Final Test MAE: {test_mae_best}")
print(f"Final Test RMSE: {test_rmse_best}")
print(f"Final Test R²: {test_r2_best}")

## END